# Padé appproximation

In [1]:
import logging

import numpy

import cicada
from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator

from statistics import mean, stdev
from tqdm import *

logging.basicConfig(level=logging.INFO)

def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = AdditiveProtocolSuite(communicator)

    values = numpy.array(1)
    log.info(f"Player {communicator.rank} values: {values}")

    values_share = protocol.share(src=0, secret=values, shape=())
    tay_tanh_share = protocol.taylor_approx(numpy.tanh, values_share)
    tay_tanh = protocol.reveal(tay_tanh_share)
    log.info(f"Player {communicator.rank} tay_tanh({values}): {tay_tanh} vs tanh({values}): {numpy.tanh(values)}")
    err_percent_list = []
    for i in tqdm(numpy.arange(-1, 1, .1)):
        cic_tay_tanh = protocol.reveal(protocol.pade_approx(numpy.tanh, protocol.share(src=0, secret=numpy.array(i), shape=()), degree=10))
        legit_tanh = numpy.tanh(i)
        err_percent_list.append(abs(legit_tanh-cic_tay_tanh))

    
    if communicator.rank ==1:
        print(mean(err_percent_list), stdev(err_percent_list))

SocketCommunicator.run(world_size=3, fn=main);

INFO:root:Player 0 values: 1
INFO:root:Player 1 values: 1
INFO:root:Player 2 values: 1
INFO:root:Player 0 tay_tanh(1): 0.7615814208984375 vs tanh(1): 0.7615941559557649
INFO:root:Player 1 tay_tanh(1): 0.7615814208984375 vs tanh(1): 0.7615941559557649
INFO:root:Player 2 tay_tanh(1): 0.7615814208984375 vs tanh(1): 0.7615941559557649
  0%|                                                                                            | 0/20 [00:00<?, ?it/s]


ERROR:cicada.communicator.socket:********************************************************************************
ERROR:cicada.communicator.socket:Comm world player 0 traceback:
ERROR:cicada.communicator.socket:Traceback (most recent call last):
  File "/home/kgoss/git/cicada-mpc/cicada/communicator/socket/__init__.py", line 784, in launch
    result = fn(communicator, *args, **kwargs)
  File "/tmp/ipykernel_4124/3865692939.py", line 27, in main
    cic_tay_tanh = protocol.reveal(protocol.pade_approx(numpy.tanh, protocol.share(src=0, secr